## Load the data and save 

In [1]:
import json
import os
from rdflib import Graph, Namespace, RDF, Literal, XSD

# Load JSON data
with open("raw-data.json", "r") as f:
    data = json.load(f)

# Define the namespace
CO_LEARNING = Namespace("http://example.org/co_learning#")

# Create the output directory if it doesn't exist
output_dir = "./rdf-data/"
os.makedirs(output_dir, exist_ok=True)

# Iterate over each CP in the data
for cp in data:
    cp_num = cp["cp_num"]
    cp_uri = CO_LEARNING[f"CP{cp_num}"]

    # Create a new RDF graph for each CP
    g = Graph()
    g.bind("co_learning", CO_LEARNING)

    # Add the CollaborationPattern instance
    g.add((cp_uri, RDF.type, CO_LEARNING.CollaborationPattern))
    g.add((cp_uri, CO_LEARNING.hasCPNum, Literal(cp["cp_num"], datatype=XSD.integer)))
    g.add(
        (
            cp_uri,
            CO_LEARNING.hasParticipantNumber,
            Literal(cp["participant"], datatype=XSD.integer),
        )
    )
    g.add(
        (
            cp_uri,
            CO_LEARNING.hasRoundNumber,
            Literal(cp["round_num"], datatype=XSD.integer),
        )
    )
    g.add(
        (
            cp_uri,
            CO_LEARNING.hasTimeScore,
            Literal(cp["time_score"], datatype=XSD.integer),
        )
    )

    # Add the new properties
    # hasCPLabel
    cp_label = cp.get("cp_name", "")
    g.add((cp_uri, CO_LEARNING.hasCPLabel, Literal(cp_label, datatype=XSD.string)))

    # hasTicksLasted
    ticks_lasted = cp.get("ticks_lasted", 0)
    g.add(
        (
            cp_uri,
            CO_LEARNING.hasTicksLasted,
            Literal(ticks_lasted, datatype=XSD.integer),
        )
    )

    # Handle situations
    prev_situation_uri = None
    for idx, situation in enumerate(cp["situation"]):
        situation_uri = CO_LEARNING[f"Situation{cp_num}_{idx}"]
        g.add((situation_uri, RDF.type, CO_LEARNING.Situation))

        if idx == 0:
            g.add((cp_uri, CO_LEARNING.hasSituation, situation_uri))
        else:
            g.add((prev_situation_uri, CO_LEARNING.hasNextSituation, situation_uri))
        prev_situation_uri = situation_uri

        # Add data properties to Situation
        for elem in situation:
            elem_type = elem["type"]
            elem_content = elem["content"]

            if elem_type == "actor":
                g.add((situation_uri, CO_LEARNING.hasActor, Literal(elem_content)))
            elif elem_type == "action":
                g.add((situation_uri, CO_LEARNING.hasAction, Literal(elem_content)))
            elif elem_type == "location":
                g.add((situation_uri, CO_LEARNING.hasLocation, Literal(elem_content)))
            elif elem_type == "object":
                g.add((situation_uri, CO_LEARNING.hasObject, Literal(elem_content)))

    # Handle ActionHuman
    prev_action_human_uri = None
    for idx, action_human in enumerate(cp["actionHuman"]):
        action_human_uri = CO_LEARNING[f"ActionHuman{cp_num}_{idx}"]
        g.add((action_human_uri, RDF.type, CO_LEARNING.ActionHuman))

        if idx == 0:
            g.add((cp_uri, CO_LEARNING.hasActionHuman, action_human_uri))
        else:
            g.add(
                (
                    prev_action_human_uri,
                    CO_LEARNING.hasNextActionHuman,
                    action_human_uri,
                )
            )
        prev_action_human_uri = action_human_uri

        # Add data properties to ActionHuman
        for elem in action_human:
            elem_type = elem["type"]
            elem_content = elem["content"]
            if elem_type == "actor":
                g.add((action_human_uri, CO_LEARNING.hasActor, Literal(elem_content)))
            elif elem_type == "action":
                g.add((action_human_uri, CO_LEARNING.hasAction, Literal(elem_content)))
            elif elem_type == "location":
                g.add(
                    (action_human_uri, CO_LEARNING.hasLocation, Literal(elem_content))
                )
            elif elem_type == "object":
                g.add((action_human_uri, CO_LEARNING.hasObject, Literal(elem_content)))

    # Handle ActionRobot
    prev_action_robot_uri = None
    for idx, action_robot in enumerate(cp["actionRobot"]):
        action_robot_uri = CO_LEARNING[f"ActionRobot{cp_num}_{idx}"]
        g.add((action_robot_uri, RDF.type, CO_LEARNING.ActionRobot))

        if idx == 0:
            g.add((cp_uri, CO_LEARNING.hasActionRobot, action_robot_uri))
        else:
            g.add(
                (
                    prev_action_robot_uri,
                    CO_LEARNING.hasNextActionRobot,
                    action_robot_uri,
                )
            )
        prev_action_robot_uri = action_robot_uri

        # Add data properties to ActionRobot
        for elem in action_robot:
            elem_type = elem["type"]
            elem_content = elem["content"]
            if elem_type == "actor":
                g.add((action_robot_uri, CO_LEARNING.hasActor, Literal(elem_content)))
            elif elem_type == "action":
                g.add((action_robot_uri, CO_LEARNING.hasAction, Literal(elem_content)))
            elif elem_type == "location":
                g.add(
                    (action_robot_uri, CO_LEARNING.hasLocation, Literal(elem_content))
                )
            elif elem_type == "object":
                g.add((action_robot_uri, CO_LEARNING.hasObject, Literal(elem_content)))

    # Format cp_num with leading zeros to three digits
    cp_num_formatted = f"{cp_num:03d}"

    # Serialize each CP's graph to a separate Turtle file named '<cp_num_formatted>.ttl'
    file_name = os.path.join(output_dir, f"{cp_num_formatted}.ttl")
    g.serialize(destination=file_name, format="turtle")
    print(f"Saved CP{cp_num} to {file_name}")

Saved CP0 to ./rdf-data/000.ttl
Saved CP1 to ./rdf-data/001.ttl
Saved CP2 to ./rdf-data/002.ttl
Saved CP3 to ./rdf-data/003.ttl
Saved CP4 to ./rdf-data/004.ttl
Saved CP5 to ./rdf-data/005.ttl
Saved CP6 to ./rdf-data/006.ttl
Saved CP7 to ./rdf-data/007.ttl
Saved CP8 to ./rdf-data/008.ttl
Saved CP9 to ./rdf-data/009.ttl
Saved CP10 to ./rdf-data/010.ttl
Saved CP11 to ./rdf-data/011.ttl
Saved CP12 to ./rdf-data/012.ttl
Saved CP13 to ./rdf-data/013.ttl
Saved CP14 to ./rdf-data/014.ttl
Saved CP15 to ./rdf-data/015.ttl
Saved CP16 to ./rdf-data/016.ttl
Saved CP17 to ./rdf-data/017.ttl
Saved CP18 to ./rdf-data/018.ttl
Saved CP19 to ./rdf-data/019.ttl
Saved CP20 to ./rdf-data/020.ttl
Saved CP21 to ./rdf-data/021.ttl
Saved CP22 to ./rdf-data/022.ttl
Saved CP23 to ./rdf-data/023.ttl
Saved CP24 to ./rdf-data/024.ttl
Saved CP25 to ./rdf-data/025.ttl
Saved CP26 to ./rdf-data/026.ttl
Saved CP27 to ./rdf-data/027.ttl
Saved CP28 to ./rdf-data/028.ttl
Saved CP29 to ./rdf-data/029.ttl
Saved CP30 to ./rdf-